# Importação e preparação dos dados

In [1]:
import pandas as pd

df = pd.read_csv('../dados/excertos_filtrados1000.csv', sep = '|')
df.head()

,COD,DESCR_AREA,filtrado
0,1400,Responsabilidade,voto : cuidar o auto de tomada de conta especi...
1,1700,Finanças Públicas,voto : cuidar o auto de solicitação de o congr...
2,5700,Responsabilidade,relatório : tratar se de embargo de declaração...
3,284,Direito Processual,voto : em relação a outro processo judiciais q...
4,298,Pessoal,": a o ato o senhor , rememorar , desde logo , ..."


In [2]:
df.shape

(13285, 3)

In [3]:
df.groupby(['DESCR_AREA']).size()

DESCR_AREA
Competência do TCU          553
Contrato Administrativo     941
Convênio                    683
Desestatização              139
Direito Processual         1811
Finanças Públicas           328
Gestão Administrativa       338
Licitação                  2756
Pessoal                    3393
Responsabilidade           2343
dtype: int64

In [4]:
areas = df.groupby(['DESCR_AREA']).groups.keys()
areas

dict_keys(['Competência do TCU', 'Contrato Administrativo', 'Convênio', 'Desestatização', 'Direito Processual', 'Finanças Públicas', 'Gestão Administrativa', 'Licitação', 'Pessoal', 'Responsabilidade'])

In [5]:
from sklearn.preprocessing import LabelBinarizer

lbArea = LabelBinarizer()
lbArea.fit([x for x in areas])
lbArea.classes_

array(['Competência do TCU', 'Contrato Administrativo', 'Convênio',
       'Desestatização', 'Direito Processual', 'Finanças Públicas',
       'Gestão Administrativa', 'Licitação', 'Pessoal',
       'Responsabilidade'], dtype='<U23')

In [6]:
y = lbArea.transform(df['DESCR_AREA'])
y.shape

(13285, 10)

In [11]:
from keras.preprocessing.text import Tokenizer
import numpy as np

vocabulario = 30000
limite_texto = 1000
dim_vetor = 50

tokenizer = Tokenizer(num_words=vocabulario)
tokenizer.fit_on_texts(df['filtrado'].astype(str))

sequences = tokenizer.texts_to_sequences(df['filtrado'].astype(str))

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 21578 unique tokens.


In [12]:
vocabulario = 21578
max = 0
lens = []
for seq in sequences:
    lens.append(len(seq))
np.min(lens), np.mean(lens), np.max(lens), np.std(lens), len(lens), sum(pd.Series(lens) > 2000)

(1, 494.97041776439596, 1426, 241.48463046337437, 13285, 0)

In [13]:
from keras.preprocessing.sequence import pad_sequences

x = pad_sequences(sequences, maxlen=limite_texto)

print('Shape of data tensor:', x.shape)

Shape of data tensor: (13285, 1000)


In [14]:
x.shape, y.shape

((13285, 1000), (13285, 10))

# Treinamento

In [15]:
from keras.models import Sequential
from keras.layers import SimpleRNN, Dense, Embedding

model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=x.shape[1]))
model.add(SimpleRNN(256))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adadelta',  metrics=["categorical_accuracy"])
model.summary()

W1030 10:57:36.270092 140366214534976 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1030 10:57:36.290893 140366214534976 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1030 10:57:36.294917 140366214534976 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1030 10:57:36.685750 140366214534976 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1000, 50)          1078900   
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 256)               78592     
_________________________________________________________________
dense_1 (Dense)              (None, 10)                2570      
Total params: 1,160,062
Trainable params: 1,160,062
Non-trainable params: 0
_________________________________________________________________


In [16]:
history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

W1030 10:57:44.564824 140366214534976 deprecation.py:323] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W1030 10:57:44.737893 140366214534976 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 111s 10ms/step - loss: 1.9752 - categorical_accuracy: 0.2780 - val_loss: 2.7178 - val_categorical_accuracy: 0.1712
Epoch 2/20
10628/10628 [==============================] - 111s 10ms/step - loss: 1.8312 - categorical_accuracy: 0.3463 - val_loss: 1.9626 - val_categorical_accuracy: 0.2872
Epoch 3/20
10628/10628 [==============================] - 110s 10ms/step - loss: 1.7767 - categorical_accuracy: 0.3800 - val_loss: 1.8299 - val_categorical_accuracy: 0.3447
Epoch 4/20
10628/10628 [==============================] - 111s 10ms/step - loss: 1.7423 - categorical_accuracy: 0.4016 - val_loss: 2.0707 - val_categorical_accuracy: 0.3037
Epoch 5/20
10628/10628 [==============================] - 111s 10ms/step - loss: 1.7786 - categorical_accuracy: 0.3782 - val_loss: 2.5517 - val_categorical_accuracy: 0.1633
Epoch 6/20
10628/10628 [==============================] - 111s 10ms/step - loss: 1.888

In [19]:
from keras.layers import GRU

model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=x.shape[1]))
model.add(GRU(1024))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 1000, 50)          1078900   
_________________________________________________________________
gru_2 (GRU)                  (None, 1024)              3302400   
_________________________________________________________________
dense_3 (Dense)              (None, 10)                10250     
Total params: 4,391,550
Trainable params: 4,391,550
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 328s 31ms/step - loss: 1.6671 - categorical_accuracy: 0.4394 - val_loss: 1.5772 - val_categorical_accuracy: 0.4784
Epoch 2/20
10628/10628 [==============================] - 325s 31ms/step - loss: 2.1280 - categorical_accuracy: 0.2944 - val_loss: 2.0447 - val_categorical_accuracy: 0.1558
E

In [21]:
from keras.layers import LSTM

model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=x.shape[1]))
model.add(LSTM(256))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 1000, 50)          1078900   
_________________________________________________________________
lstm_2 (LSTM)                (None, 256)               314368    
_________________________________________________________________
dense_5 (Dense)              (None, 10)                2570      
Total params: 1,395,838
Trainable params: 1,395,838
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 357s 34ms/step - loss: 1.6782 - categorical_accuracy: 0.4287 - val_loss: 1.6259 - val_categorical_accuracy: 0.4419
Epoch 2/20
10628/10628 [==============================] - 355s 33ms/step - loss: 1.2413 - categorical_accuracy: 0.6080 - val_loss: 1.4795 - val_categorical_accuracy: 0.5107
E

In [22]:
from keras.layers import Bidirectional

model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=x.shape[1]))
model.add(Bidirectional(SimpleRNN(256)))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 1000, 50)          1078900   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 512)               157184    
_________________________________________________________________
dense_6 (Dense)              (None, 10)                5130      
Total params: 1,241,214
Trainable params: 1,241,214
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 215s 20ms/step - loss: 1.9675 - categorical_accuracy: 0.2834 - val_loss: 1.9717 - val_categorical_accuracy: 0.1935
Epoch 2/20
10628/10628 [==============================] - 213s 20ms/step - loss: 1.6882 - categorical_accuracy: 0.4318 - val_loss: 1.8549 - val_categorical_accuracy: 0.3432
E

In [23]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=x.shape[1]))
model.add(Bidirectional(GRU(256)))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 1000, 50)          1078900   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 512)               471552    
_________________________________________________________________
dense_7 (Dense)              (None, 10)                5130      
Total params: 1,555,582
Trainable params: 1,555,582
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 588s 55ms/step - loss: 1.7158 - categorical_accuracy: 0.4218 - val_loss: 1.5773 - val_categorical_accuracy: 0.4678
Epoch 2/20
10628/10628 [==============================] - 584s 55ms/step - loss: 1.4083 - categorical_accuracy: 0.5547 - val_loss: 1.6094 - val_categorical_accuracy: 0.4626
E

In [24]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=x.shape[1]))
model.add(Bidirectional(LSTM(256)))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 1000, 50)          1078900   
_________________________________________________________________
bidirectional_3 (Bidirection (None, 512)               628736    
_________________________________________________________________
dense_8 (Dense)              (None, 10)                5130      
Total params: 1,712,766
Trainable params: 1,712,766
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 642s 60ms/step - loss: 1.7023 - categorical_accuracy: 0.4196 - val_loss: 1.7738 - val_categorical_accuracy: 0.3515
Epoch 2/20
10628/10628 [==============================] - 640s 60ms/step - loss: 1.3748 - categorical_accuracy: 0.5609 - val_loss: 1.7349 - val_categorical_accuracy: 0.4426
E

In [25]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=x.shape[1]))
model.add(GRU(128))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 1000, 50)          1078900   
_________________________________________________________________
gru_4 (GRU)                  (None, 128)               68736     
_________________________________________________________________
dense_9 (Dense)              (None, 10)                1290      
Total params: 1,148,926
Trainable params: 1,148,926
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 284s 27ms/step - loss: 1.7207 - categorical_accuracy: 0.4056 - val_loss: 1.8882 - val_categorical_accuracy: 0.3798
Epoch 2/20
10628/10628 [==============================] - 282s 27ms/step - loss: 1.1481 - categorical_accuracy: 0.6202 - val_loss: 1.3798 - val_categorical_accuracy: 0.4991
E

In [26]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=x.shape[1]))
model.add(GRU(64))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 1000, 50)          1078900   
_________________________________________________________________
gru_5 (GRU)                  (None, 64)                22080     
_________________________________________________________________
dense_10 (Dense)             (None, 10)                650       
Total params: 1,101,630
Trainable params: 1,101,630
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 280s 26ms/step - loss: 1.7692 - categorical_accuracy: 0.3904 - val_loss: 1.6747 - val_categorical_accuracy: 0.3933
Epoch 2/20
10628/10628 [==============================] - 278s 26ms/step - loss: 1.1641 - categorical_accuracy: 0.6143 - val_loss: 1.6752 - val_categorical_accuracy: 0.5058
E

In [27]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=x.shape[1]))
model.add(GRU(512))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 1000, 50)          1078900   
_________________________________________________________________
gru_6 (GRU)                  (None, 512)               864768    
_________________________________________________________________
dense_11 (Dense)             (None, 10)                5130      
Total params: 1,948,798
Trainable params: 1,948,798
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 286s 27ms/step - loss: 1.7146 - categorical_accuracy: 0.4192 - val_loss: 1.7874 - val_categorical_accuracy: 0.4091
Epoch 2/20
10628/10628 [==============================] - 286s 27ms/step - loss: 1.2286 - categorical_accuracy: 0.6005 - val_loss: 1.4213 - val_categorical_accuracy: 0.5149
E

In [28]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=x.shape[1]))
model.add(GRU(256))
model.add(Dense(128, activation='relu'))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (None, 1000, 50)          1078900   
_________________________________________________________________
gru_7 (GRU)                  (None, 256)               235776    
_________________________________________________________________
dense_12 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_13 (Dense)             (None, 10)                1290      
Total params: 1,348,862
Trainable params: 1,348,862
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 287s 27ms/step - loss: 1.6458 - categorical_accuracy: 0.4355 - val_loss: 1.5226 - val_categorical_accuracy: 0.4915
Epoch 2/20
10628/10628 [==================

In [29]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=x.shape[1]))
model.add(GRU(256))
model.add(Dense(64, activation='relu'))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_13 (Embedding)     (None, 1000, 50)          1078900   
_________________________________________________________________
gru_8 (GRU)                  (None, 256)               235776    
_________________________________________________________________
dense_14 (Dense)             (None, 64)                16448     
_________________________________________________________________
dense_15 (Dense)             (None, 10)                650       
Total params: 1,331,774
Trainable params: 1,331,774
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 281s 26ms/step - loss: 1.7479 - categorical_accuracy: 0.3891 - val_loss: 1.7399 - val_categorical_accuracy: 0.4196
Epoch 2/20
10628/10628 [==================

In [30]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=x.shape[1]))
model.add(GRU(256))
model.add(Dense(32, activation='relu'))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_14 (Embedding)     (None, 1000, 50)          1078900   
_________________________________________________________________
gru_9 (GRU)                  (None, 256)               235776    
_________________________________________________________________
dense_16 (Dense)             (None, 32)                8224      
_________________________________________________________________
dense_17 (Dense)             (None, 10)                330       
Total params: 1,323,230
Trainable params: 1,323,230
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 282s 27ms/step - loss: 1.6353 - categorical_accuracy: 0.4358 - val_loss: 1.6723 - val_categorical_accuracy: 0.4513
Epoch 2/20
10628/10628 [==================

In [31]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=x.shape[1]))
model.add(GRU(256, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

W1031 10:34:08.570308 140366214534976 deprecation.py:506] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_15 (Embedding)     (None, 1000, 50)          1078900   
_________________________________________________________________
gru_10 (GRU)                 (None, 256)               235776    
_________________________________________________________________
dense_18 (Dense)             (None, 10)                2570      
Total params: 1,317,246
Trainable params: 1,317,246
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 325s 31ms/step - loss: 1.6914 - categorical_accuracy: 0.4213 - val_loss: 1.5354 - val_categorical_accuracy: 0.5322
Epoch 2/20
10628/10628 [==============================] - 322s 30ms/step - loss: 1.1892 - categorical_accuracy: 0.6098 - val_loss: 1.4187 - val_categorical_accuracy: 0.4900
E

In [32]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=x.shape[1]))
model.add(GRU(256, dropout=0.1, recurrent_dropout=0.5))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_16 (Embedding)     (None, 1000, 50)          1078900   
_________________________________________________________________
gru_11 (GRU)                 (None, 256)               235776    
_________________________________________________________________
dense_19 (Dense)             (None, 10)                2570      
Total params: 1,317,246
Trainable params: 1,317,246
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 400s 38ms/step - loss: 1.7395 - categorical_accuracy: 0.3950 - val_loss: 1.6265 - val_categorical_accuracy: 0.4434
Epoch 2/20
10628/10628 [==============================] - 397s 37ms/step - loss: 1.2488 - categorical_accuracy: 0.5949 - val_loss: 1.7142 - val_categorical_accuracy: 0.4569
E

In [33]:
from keras.optimizers import RMSprop

model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=x.shape[1]))
model.add(GRU(256))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_17 (Embedding)     (None, 1000, 50)          1078900   
_________________________________________________________________
gru_12 (GRU)                 (None, 256)               235776    
_________________________________________________________________
dense_20 (Dense)             (None, 10)                2570      
Total params: 1,317,246
Trainable params: 1,317,246
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 339s 32ms/step - loss: 1.7194 - categorical_accuracy: 0.4147 - val_loss: 1.6092 - val_categorical_accuracy: 0.4403
Epoch 2/20
10628/10628 [==============================] - 337s 32ms/step - loss: 1.2310 - categorical_accuracy: 0.6076 - val_loss: 1.5709 - val_categorical_accuracy: 0.4268
E

In [34]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=x.shape[1]))
model.add(GRU(256))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adadelta',  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_18 (Embedding)     (None, 1000, 50)          1078900   
_________________________________________________________________
gru_13 (GRU)                 (None, 256)               235776    
_________________________________________________________________
dense_21 (Dense)             (None, 10)                2570      
Total params: 1,317,246
Trainable params: 1,317,246
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 349s 33ms/step - loss: 1.8284 - categorical_accuracy: 0.3464 - val_loss: 1.8765 - val_categorical_accuracy: 0.3165
Epoch 2/20
10628/10628 [==============================] - 346s 33ms/step - loss: 1.4664 - categorical_accuracy: 0.5182 - val_loss: 1.6124 - val_categorical_accuracy: 0.4400
E

In [35]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=x.shape[1]))
model.add(GRU(256, return_sequences=True))
model.add(GRU(32))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_19 (Embedding)     (None, 1000, 50)          1078900   
_________________________________________________________________
gru_14 (GRU)                 (None, 1000, 256)         235776    
_________________________________________________________________
gru_15 (GRU)                 (None, 32)                27744     
_________________________________________________________________
dense_22 (Dense)             (None, 10)                330       
Total params: 1,342,750
Trainable params: 1,342,750
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 668s 63ms/step - loss: 1.9057 - categorical_accuracy: 0.3481 - val_loss: 1.9306 - val_categorical_accuracy: 0.2364
Epoch 2/20
10628/10628 [==================

In [36]:
from keras.layers.core import Dropout

model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=x.shape[1]))
model.add(GRU(256, dropout=0.1, recurrent_dropout=0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_20 (Embedding)     (None, 1000, 50)          1078900   
_________________________________________________________________
gru_16 (GRU)                 (None, 256)               235776    
_________________________________________________________________
dense_23 (Dense)             (None, 128)               32896     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_24 (Dense)             (None, 10)                1290      
Total params: 1,348,862
Trainable params: 1,348,862
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 331s 31ms/step - loss: 1.

In [37]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=x.shape[1]))
model.add(GRU(256))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=64, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_21 (Embedding)     (None, 1000, 50)          1078900   
_________________________________________________________________
gru_17 (GRU)                 (None, 256)               235776    
_________________________________________________________________
dense_25 (Dense)             (None, 10)                2570      
Total params: 1,317,246
Trainable params: 1,317,246
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 145s 14ms/step - loss: 1.8330 - categorical_accuracy: 0.3551 - val_loss: 1.8391 - val_categorical_accuracy: 0.3128
Epoch 2/20
10628/10628 [==============================] - 143s 13ms/step - loss: 1.5455 - categorical_accuracy: 0.4999 - val_loss: 1.5700 - val_categorical_accuracy: 0.4682
E

In [38]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=x.shape[1]))
model.add(GRU(256, dropout=0.1, recurrent_dropout=0.5))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=64, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_22 (Embedding)     (None, 1000, 50)          1078900   
_________________________________________________________________
gru_18 (GRU)                 (None, 256)               235776    
_________________________________________________________________
dense_26 (Dense)             (None, 10)                2570      
Total params: 1,317,246
Trainable params: 1,317,246
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 166s 16ms/step - loss: 1.8214 - categorical_accuracy: 0.3613 - val_loss: 1.7445 - val_categorical_accuracy: 0.3914
Epoch 2/20
10628/10628 [==============================] - 163s 15ms/step - loss: 1.4689 - categorical_accuracy: 0.5204 - val_loss: 1.8327 - val_categorical_accuracy: 0.3782
E

In [39]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=x.shape[1]))
model.add(GRU(1024, dropout=0.1, recurrent_dropout=0.5))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=64, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_23 (Embedding)     (None, 1000, 50)          1078900   
_________________________________________________________________
gru_19 (GRU)                 (None, 1024)              3302400   
_________________________________________________________________
dense_27 (Dense)             (None, 10)                10250     
Total params: 4,391,550
Trainable params: 4,391,550
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 239s 22ms/step - loss: 1.9435 - categorical_accuracy: 0.3561 - val_loss: 1.8741 - val_categorical_accuracy: 0.3474
Epoch 2/20
10628/10628 [==============================] - 236s 22ms/step - loss: 1.6082 - categorical_accuracy: 0.4937 - val_loss: 1.6488 - val_categorical_accuracy: 0.4343
E

In [41]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=x.shape[1]))
model.add(Bidirectional(GRU(256, dropout=0.1, recurrent_dropout=0.4)))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_24 (Embedding)     (None, 1000, 50)          1078900   
_________________________________________________________________
bidirectional_4 (Bidirection (None, 512)               471552    
_________________________________________________________________
dense_28 (Dense)             (None, 10)                5130      
Total params: 1,555,582
Trainable params: 1,555,582
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 666s 63ms/step - loss: 1.7079 - categorical_accuracy: 0.4230 - val_loss: 1.7644 - val_categorical_accuracy: 0.3007
Epoch 2/20
10628/10628 [==============================] - 631s 59ms/step - loss: 1.4539 - categorical_accuracy: 0.5246 - val_loss: 1.8625 - val_categorical_accuracy: 0.4234
E